# Read NPPES Raw

In [ ]:
# from sqlalchemy import create_engine
# import pandas as pd
# db_connection_str = 'mysql+pymysql://root:@localhost:3306/physicianinfo?charset=utf8'
# db_connection = create_engine(db_connection_str)
# sql = 'SELECT * FROM physicianinfo.NPPESPhysician where MDDO = 1;'
# df = pd.read_sql(sql, con=db_connection)
# df.to_pickle('Data/NPPES/NPPESMDDOPhysician.p')
# print(len(df))

import pandas as pd
df = pd.read_pickle('Data/NPPES/NPPESMDDOPhysician.p')
print(len(df))
df.head()

# Zip Information

In [ ]:
import numpy as np
import json

def get_zip_code(x):
    try: 
        zipcode = int(json.loads(x)["Postal Code"]) 
    except: 
        zipcode = 0
    return zipcode

df['BusinessZip'] = df['BusinessMailingAddress'].apply(lambda x: get_zip_code(x))# .astype(int)
df['PracticeZip'] = df['PracticeLocationAddress'].apply(lambda x: get_zip_code(x))# .astype(int)
df['BusinessZip5'] = df['BusinessZip'].apply(lambda x: int(str(x)[:5]) if len(str(x)) in [5, 9] else np.nan)
df['PracticeZip5'] = df['PracticeZip'].apply(lambda x: int(str(x)[:5]) if len(str(x)) in [5, 9] else np.nan)

df.columns

# Specialty Information

In [ ]:
path = 'Data/NPPES/nucc_taxonomy_211.csv'
RawData = pd.read_csv(path)

code2specialty = RawData[['Code', 'Display Name']]
code2specialty_dict = dict(zip(code2specialty['Code'].values, code2specialty['Display Name'].values))
df['Specialty'] = df['MainTaxonomy'].apply(lambda x: code2specialty_dict.get(x, None))

# Graduation Year

In [ ]:
path = 'Data/DAC_NationalDownloadableFile.csv'

RawData = pd.read_csv(path, encoding='ISO-8859-1', low_memory=False)
RawData.columns = [i.strip() for i in RawData.columns]
RawData = RawData[- RawData['NPI'].duplicated()].reset_index(drop = True)
tmp = pd.merge(df[['NPI', 'LastName', 'FirstName', 'Gender', 'Credential', 'Sole', 
                   'BusinessZip', 'PracticeZip', 'Specialty', 'BusinessZip5', 'PracticeZip5']], 
               RawData[['NPI', 'Med_sch', 'Grd_yr', 'pri_spec', 'zip']],  on = 'NPI', how = 'left')
tmp

In [ ]:
tmp[['NPI', 'LastName', 'FirstName']].to_csv('Output/NPI2Name.csv', index = False)

# Race

In [ ]:
path = 'Data/NPPES/NPI2NameMDDORace.csv'
cols = ['NPI', 'race', 'asian', 'hispanic', 'nh_black', 'nh_white']
NPI2NameRace = pd.read_csv(path)[cols]
FinalData = pd.merge(tmp, NPI2NameRace, on = 'NPI', how = 'left')
FinalData

# Zip Information

In [ ]:
zipSeries = pd.Series(list(set(FinalData['BusinessZip'].to_list() + FinalData['PracticeZip'].to_list())))
zipSeries.apply(lambda x: len(str(x))).value_counts().sort_index()

In [ ]:
zipcode2freq = zipSeries.apply(lambda x: str(x)[:5] if len(str(x)) in [5, 9] else np.nan).value_counts().sort_index()
zipcode2freq

In [ ]:
import pandas as pd
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

L = []
for zipcode in zipcode2freq.index:
    zipcode = str(zipcode)
    # print(zipcode)
    
    d = {}
    try:
        info = search.by_zipcode(zipcode).to_dict()
        d['zipcode'] = zipcode
        d['population'] = info['population']
        d['population_density'] = info['population_density']
        L.append(d)
    except:
        print(zipcode)

Zip2Population = pd.DataFrame(L)
Zip2Population.to_pickle('Output/Zip2Population.p')
Zip2Population